In [ ]:
# !pip uninstall torch torchvision torchaudio -y
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 

In [ ]:
!pip install roboflow opencv-python matplotlib ultralytics

In [ ]:
import torch

print("Torch version:", torch.__version__)
print("CUDA version:", torch.version.cuda)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))


In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="puWNSw1Du2OCFEHTegQs")
project = rf.workspace("trading-al").project("project-trading-yolov5-v1")
version = project.version(3)
dataset = version.download("yolov8-obb")             

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8m.pt")  # Model trung bình, chính xác hơn yolov8s

model.train(
    data=r"D:\Yolov8 trading AL\project-trading-yolov5-v1--3\data.yaml",
    epochs=150,          # Cho model nhiều thời gian học
    imgsz=768,           # Hình ảnh lớn hơn => phát hiện vật thể nhỏ tốt hơn
    batch=32,            # Cân bằng giữa VRAM và hiệu quả
    workers=4,
    device=0,
    amp=True,            # Mixed precision: tăng tốc, tiết kiệm RAM
    patience=15,         # Early stopping thông minh
    lr0=0.01,
    lrf=0.01,
    project="my_model",
    name="yolov8m_highacc",
    exist_ok=True
)

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8s.pt")  # hoặc yolov8n.pt nếu máy yếu

# Huấn luyện mô hình
model.train(data="D:\Yolov8 trading AL\project-trading-yolov5-v1--3\data.yaml", epochs=100, imgsz=640, batch=32, project="my_model", name="my_yolov8", exist_ok=True)


In [ ]:
import cv2
import os
from ultralytics import YOLO

# Đường dẫn mô hình YOLOv8 đã huấn luyện
model_path = 'my_model/my_yolov8/weights/best.pt'

# Kiểm tra mô hình
if not os.path.exists(model_path):
    print("❌ Không tìm thấy file mô hình:", model_path)
    exit()

# Tải mô hình YOLOv8
model = YOLO(model_path)

# Mở webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("❌ Không thể mở webcam.")
    exit()

# Thiết lập độ phân giải 1300x731 (tỷ lệ 16:9)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1300)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 731)

while True:
    ret, frame = cap.read()
    if not ret:
        print("❌ Không thể đọc từ webcam.")
        break

    # Suy luận
    results = model.predict(source=frame, imgsz=640, conf=0.3)

    # Gán kết quả vào frame đã annotate
    annotated_frame = results[0].plot()

    # Hiển thị
    cv2.imshow("YOLOv8 Detection (1920x1080)", annotated_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
